In [1]:
import pandas as pd
import numpy as np
import scipy as sp
import scipy.sparse as sps
import scipy.io as io
import time
from scipy.sparse.linalg import svds
from sklearn.preprocessing import normalize

### Read needed files

In [2]:
# read interactions file
train_final = pd.read_csv('../input/train_final.csv', sep = '\t')
train_final['interaction'] = 1.0
train_final = train_final.sort_values(['playlist_id', 'track_id'], ascending=[True, True])
numberInteractions = train_final.shape[0]
print(train_final.iloc[:20,])

        playlist_id  track_id  interaction
770769         7569    162463          1.0
925571         7569    421750          1.0
912580         7569    795606          1.0
852833         7569   1195736          1.0
287048         7569   2227105          1.0
92449          7569   2634448          1.0
519471         7569   2654612          1.0
173680         7569   2693660          1.0
903004         7569   2861386          1.0
863056         7614    415173          1.0
278077         7614   1384962          1.0
394480         7614   1609224          1.0
615615         7614   1614974          1.0
328664         7614   1714787          1.0
377085         7614   2141817          1.0
83840          7614   2285204          1.0
574199         7614   3361942          1.0
481594         7614   3504896          1.0
300753         7614   3711434          1.0
775317         7614   3833025          1.0


We seperate each column in a seperate list:

In [3]:
playlist_id = list(train_final['playlist_id'])
track_id = list(train_final['track_id'])
interaction = list(train_final['interaction'])
print(playlist_id[:10])
print(track_id[:10])
print(interaction[:10])

[7569, 7569, 7569, 7569, 7569, 7569, 7569, 7569, 7569, 7614]
[162463, 421750, 795606, 1195736, 2227105, 2634448, 2654612, 2693660, 2861386, 415173]
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]


Create index mapping for playlist and track IDs

In [4]:
playlist_id_unique = list(set(playlist_id))
track_id_unique = list(set(track_id))
playlist_id_unique.sort()
playlist_id_unique = pd.DataFrame(playlist_id_unique)
playlist_id_unique.reset_index(level=0, inplace=True)
playlist_id_unique.columns = ['index_playlist', 'playlist_id']
track_id_unique.sort()
track_id_unique = pd.DataFrame(track_id_unique)
track_id_unique.reset_index(level=0, inplace=True)
track_id_unique.columns = ['index_track', 'track_id']
print(track_id_unique.head())
print(playlist_id_unique.head())

   index_track  track_id
0            0       252
1            1       360
2            2       394
3            3       527
4            4       667
   index_playlist  playlist_id
0               0         7569
1               1         7614
2               2         7641
3               3         7692
4               4         7816


In [5]:
numPlaylists = len(playlist_id_unique)
numTracks = len(track_id_unique)

In [6]:
train_intermediate = train_final.merge(playlist_id_unique, how='inner', on='playlist_id')
train_index = train_intermediate.merge(track_id_unique, how='inner', on='track_id')
train_index = train_index.sort_values(['playlist_id', 'track_id'], ascending=[True, True])
train_index

,playlist_id,track_id,interaction,index_playlist,index_track
0,7569,162463,1.0,0,3484
87,7569,421750,1.0,0,10864
116,7569,795606,1.0,0,20062
125,7569,1195736,1.0,0,29176
195,7569,2227105,1.0,0,55647
198,7569,2634448,1.0,0,66029
241,7569,2654612,1.0,0,66596
253,7569,2693660,1.0,0,67590
263,7569,2861386,1.0,0,73675
298,7614,415173,1.0,1,10624


In [7]:
index_playlist = list(train_index['index_playlist'])
index_track = list(train_index['index_track'])
interaction = list(train_index['interaction'])
print(index_playlist[:10])
print(index_track[:10])
print(interaction[:10])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
[3484, 10864, 20062, 29176, 55647, 66029, 66596, 67590, 73675, 10624]
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]


In [8]:
URM_all = sps.coo_matrix((interaction, (index_playlist, index_track)))
URM_all.tocsr()
URM_all

<45649x99999 sparse matrix of type '<class 'numpy.float64'>'
	with 1040522 stored elements in COOrdinate format>

In [ ]:
URM = np.zeros(shape=(45649,99999))
print(URM.shape)
print(URM[0,2])

In [ ]:
start_time = time.time()
for index, row in train_index.iterrows():
    URM[row['index_playlist'], row['index_track']] = row['interaction']
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
mean_URM = np.mean(URM, axis = 1)

In [ ]:
mean_URM[:10]
mean_URM.shape

In [ ]:
demeaned_URM = URM - mean_URM.reshape(-1, 1)

In [ ]:
# taking too long
start_time = time.time()
U, sigma, Vt = svds(demeaned_URM, k = 50)
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# kernel dies because matrix is not sparse anymore
demeaned_URM_all = URM_all - URM_all.mean(axis = 1)

In [9]:
start_time = time.time()
U, s, Vt = svds(URM_all, k = 50)
print("--- %s seconds ---" % (time.time() - start_time))

--- 4.0392842292785645 seconds ---


In [10]:
s_diag = np.diag(s)

In [11]:
Us = np.dot(U, s_diag)
Us.shape

(45649, 50)

In [12]:
start_time = time.time()
URM_estm = np.dot(Us, Vt)
print("--- %s seconds ---" % (time.time() - start_time))

--- 122.29415702819824 seconds ---


In [15]:
print(URM_estm.shape)
print(URM_estm[:10,:10])
print(URM_estm[0,:].max())

(45649, 99999)
[[  5.81494202e-05   1.77410812e-05  -2.46214609e-04   6.01678510e-05
    5.33451296e-04   1.05360826e-04   1.13592088e-04   7.80126434e-06
    3.70764055e-05   6.80325116e-06]
 [ -3.71196522e-04   9.81899151e-04   2.83356814e-04   9.18600462e-05
   -2.30382209e-04  -3.55141076e-04  -1.74383469e-04  -2.95536379e-04
   -8.24466316e-04   8.81044619e-05]
 [  1.84603545e-04  -1.39226204e-04  -9.92982645e-05   4.61793417e-05
   -3.52985639e-05   4.18437211e-04  -6.68809021e-05   4.22801439e-04
    1.39319761e-04   5.10532905e-05]
 [  5.71655875e-05   4.66534103e-04  -1.00827194e-04  -7.25278409e-05
    2.88187117e-04   2.22843352e-04   1.19197626e-04   1.29410012e-04
    5.66197509e-05  -2.84272579e-05]
 [ -1.53325782e-04   8.49497258e-05  -1.66764096e-05   4.46740507e-05
   -1.51105629e-05  -1.36704899e-05   2.92211141e-05  -7.88016852e-05
    2.65979904e-05   4.07161985e-05]
 [  4.06218404e-03  -3.24275523e-04   7.83355342e-05  -4.93838660e-05
    4.27323937e-04   2.2617931

### Find tracks which appear in less than 5 playlists and are not in the target track set